# Issues reading SMAP with different xarray backends

This notebook explores [Issue 650](https://github.com/corteva/rioxarray/issues/650) for rioxarray.  The user finds that the y-axis is flipped when they use `xarray` with `rioxarray` to read SMAP L4 data but not when they use the `netcdf4` backend.

They use https://n5eil01u.ecs.nsidc.org/SMAP/SPL4SMGP.007/2015.03.31/SMAP_L4_SM_gph_20150331T013000_Vv7032_001.h5 as an example.

In [ ]:
from pathlib import Path
import warnings

import xarray as xr
import rioxarray
import matplotlib.pyplot as plt

print(xr.__version__)
print(rioxarray.__version__)

In [ ]:
filepath = Path("smap_data/SMAP_L4_SM_gph_20150331T013000_Vv7032_001.h5")

In [ ]:
engine = "netcdf4"
ds = xr.merge([
    xr.open_dataset(filepath, engine=engine),
    xr.open_dataset(filepath, group='Geophysical_Data', engine=engine)['precipitation_total_surface_flux'],
    ])
ds

In [ ]:
ds.precipitation_total_surface_flux.plot()

With `h5netcdf`

In [ ]:
engine="h5netcdf"
ds = xr.merge([
    xr.open_dataset(filepath, engine=engine, phony_dims="sort"),
    xr.open_dataset(filepath, group='Geophysical_Data', engine=engine, phony_dims="sort")['precipitation_total_surface_flux'],
    ])
print(ds)

In [ ]:
%%capture  
# Avoid a string of NotGeoreferencedWarnings
# /home/apbarret/mambaforge/envs/nsidc-tutorials/lib/python3.9/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: 
#  Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
#  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
engine = "rasterio"
ds_rasterio = xr.open_dataset(filepath, engine=engine)

In [ ]:
print(ds_rasterio.x, ds_rasterio.y)

In [ ]:
ds_rasterio.cell_lat.plot()
plt.savefig('smap_h5_cell_lat.png')

In [ ]:
ds_rasterio.cell_lat.squeeze().plot.imshow(origin="upper")

In [ ]:
!h5dump -d x {filepath}

In [ ]:
ds_rioxarray  = xr.merge([
    xr.open_dataset(filepath, decode_coords="all"),
    xr.open_dataset(filepath, group='Geophysical_Data', decode_coords="all")['precipitation_total_surface_flux'],
])

print(ds_rioxarray)

In [ ]:
ds_rioxarray.precipitation_total_surface_flux.plot(vmax=0.001)
plt.savefig('smap_h5_precipitation.png')

In [ ]:
ds_rioxarray.rio.crs

In [ ]:
ds_rioxarray.rio.transform()

In [ ]:
ds_rioxarray.rio.reproject?

In [ ]:
ds.cell_row.plot()

In [ ]:
ds.precipitation_total_surface_flux.plot.imshow()

Try writing a dummy geotiff and then read it back in